<a href="https://colab.research.google.com/github/hxk271/SocDataSci/blob/main/archive/W12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 12 (시각화 미세조정)

 오늘은 <b>시각화 미세조정(visualization fine-tuning)</b>에 대해 배운다. 이것은 사실 생각보다 어려운 스킬이다. 걸작을 만들어내기 위해서는 90%를 만들기보다 나머지 10% 완성하기가 훨씬 어렵다. 시간을 갈아넣은 만큼 좋은 시각화 작품이 나오고, 그러지 않은 이들의 작품은 형편없다. 예전에는 각종 노하우를 스스로 알아내기 전까지 익히기 쉽지 않았다. 그러나 요즘에는 인터넷 검색, 각종 커뮤니티, 그리고 ChatGPT 덕택에 굉장히 쉬워졌다.

In [ ]:
#오늘의 자료
import gdown
links = ['https://drive.google.com/uc?id=1oGH3n0gIAfkhqTT06xYy2BoH5LWpS070',          #ds_salary
         'https://drive.google.com/uc?id=1rpDwYGYHcbUtj2aaasObE5XEtLwwHpCv',          #ABNB_stock
         'https://drive.google.com/uc?id=1rqme8okVGSmytvVbV-i7h-ORDv7R9DYl',          #Covid19-India
         'https://drive.google.com/uc?id=1rsYj2AZ5PEuY9GMm0jIqjLCFecIbfeHg',          #CO2_emissions
         'https://drive.google.com/uc?id=1rtPJtzNwPh--WPe1r8B0CoVjncSYMGov',          #product
         'https://drive.google.com/uc?id=1ruMPohb-YZbVK9ZYgWyYaDZdiZELTJrw']          #product_inspection

for link in links:
    gdown.download(link)

## 1. 축과 레이블의 미세조정(seaborn)

> 시각화를 하다보면 축(axis) 또는 레이블(label)과 관련된 부분이 어색하고 잘 맞지 않는 경우가 많다. 이것을 조정하는 방법을 익혀보자.

In [ ]:
import pandas as pd

df = pd.read_csv('ds_salaries.csv')
df

> 실제 사례를 살펴보기 위해 연구질문을 하나 던져보자: "데이터 관련 직종에서 잡 타이틀(job title)별로 연봉이 어떻게 다를까?" 이런 문제에 대답하려면 어떤 시각화를 수행해야 할지 머리 속으로 먼저 생각해보자.
>
> 이 자료에서 잡 타이틀은 굉장히 다양하다. 그러니 일단 적어도 30개 이상의 빈도가 관찰되는 잡 타이틀만 골라내고 이것을 시각화하자. 이때 핵심은 분석을 통해 얻어낸 빈도분포표 그 자체를 또다른 자료로 받아들이는 것이다(Why?).

In [ ]:
df['job_title'].unique()
freq = df['job_title'].value_counts()
freq
type(freq)           #Series

# 30번 이상 언급된 job_title만 골라낸다
title_over_30 = freq.loc[freq>30]
title_over_30

cond = df['job_title'].isin(title_over_30.index)
df2 = df.loc[cond]

> 모든 준비가 끝났으니 먼저 seaborn으로 상자-수염 도표를 만들어보자!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
sns.boxplot(data=df2,
            x='job_title',
            y='salary_in_usd',
            ax=ax)

> 바로 이것이 가장 흔한 문제 중 하나인데, $x$축의 레이블이 서로 겹쳐 알아볼 수 없다! 이때는 $x$축 레이블을 <b>회전(rotation)</b>할 수 있다. 별 것 아니다, 그냥 $x$와 $y$를 교체하면 된다(Why?).

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df2,
            x='job_title',
            y='salary_in_usd',
            ax=ax)
ax.tick_params(axis='x', labelrotation=90)

> 회전하고 보니 차라리 상자-수염 그림을 수평으로 그리는게 나을 것 같다. 트릭은 아주 간단하다!

In [ ]:
fig, ax= plt.subplots()
sns.boxplot(data=df2,
            y='job_title',
            x='salary_in_usd',
            ax=ax)
ax.tick_params(axis='y', labelrotation=0)

ax.set(ylabel="")                     #딱보면 알 수 있으니 필요없다.
ax.set(xlabel="Salary in USD")        #쫌만 더 이쁘게

ax.xaxis.grid(True)                   #원한다면 grid 추가
ax.grid(axis='y')                     #다른 방식으로 grid 추가

> **연습문제 1-1**. 먼저 연봉(`salary`)의 평균이 가장 높은 순서대로 직종을 10개 골라내고, 다음으로 이들의 막대 차트를 그리시오. 이때 $x$축은 연봉, $y$은 잡 타이틀(`job_title`)로 설정하시오.
>
> ---
> 힌트 1: 문장을 잘 읽어보면, 골라내야 하는 단위는 직종별이다. 그러므로 직종별로 연봉 평균을 집계해야 한다(Why?).\
> 힌트 2: 막대 차트는 `seaborn.barplot()`을 사용하여 그릴 수 있다.

In [ ]:
job_salary = df.groupby('job_title')['salary_in_usd'].mean()
top10 = job_salary.sort_values(ascending = False)[0:10]
top10 = top10.reset_index()

fig, ax= plt.subplots()
sns.barplot(data=top10,
            y='job_title',
            x='salary_in_usd',
            ax=ax)
ax.tick_params(axis='y', labelrotation=0)
ax.set(ylabel="")
ax.set(xlabel="Salary in USD")
ax.xaxis.grid(True)

> 이번엔 시계열 자료(time-series data)를 연습하기 위해 에어비엔비(AirBnB) 주가 자료를 살펴보자.

In [ ]:
df = pd.read_csv('ABNB_stock.csv')
df

fig, ax = plt.subplots()
sns.lineplot(data=df,
             x='Date',
             y='Close',
             ax=ax)

 > 이때도 마찬가지로 $x$축을 식별할 수 없다(Why?). 어떻게 하면 좀 더 알아볼 수 있게 만들 수 있을까?

In [ ]:
# 변수를 살펴보자.
df.info()

# 날짜/시간 포멧으로 변경
df['Date'] = pd.to_datetime(df['Date'])

# 다시 시계열 시각화
fig, ax = plt.subplots()
sns.lineplot(data=df,
             x='Date',
             y='Close',
             ax=ax)

> 다소 특수하지만 두 개의 $y$축이 필요한 상황도 있다. 가령 에어비엔비 주가 자료에서 종가(`Close`) 시계열 뿐 아니라, 거래량(`Volume`) 시계열을 함께 그리는 경우를 생각해보자.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

fig, ax = plt.subplots(figsize=(10, 5))
ax2 = ax.twinx()            # twin x이므로 x축을 공유한다.

#첫번째 그림
sns.lineplot(data=df, x='Date', y='Close',  ax=ax,  color='red')
ax.tick_params(axis='y', labelcolor='red')
ax.yaxis.label.set_color('red')                 #좌측 y축 레이블 색상

#두번째 그림
sns.lineplot(data=df, x='Date', y='Volume', ax=ax2, color='blue')
ax2.tick_params(axis='y', labelcolor='blue')
ax2.yaxis.label.set_color('blue')               #우측 y축 레이블 색상

> **연습문제 1-2**. 고가(`High`)와 저가(`Low`) 간 격차를 계산하고 이를 조정종가(`Adj Close`)와 함께 시각화하시오.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#High-Low 계산
df = pd.read_csv('ABNB_stock.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['High-Low'] = df['High'] - df['Low']

#시각화
fig, ax = plt.subplots(figsize=(12,5))
ax2 = ax.twinx()

#종가 그림
sns.lineplot(data=df,
             x='Date', y='Adj Close',
             ax=ax,
             linestyle='--', color='black', label="Adj. Close")

#High-Low 그림
sns.lineplot(data=df,
             x='Date', y='High-Low',
             ax=ax2,
             linestyle='-', color='black', alpha=.7, linewidth = .7, label="High-Low")

#축 1번 레이블 미세조정
ax.set_ylabel("Adjusted Close Price")
ax.tick_params(axis='y')
ax.set_xlabel("Date")

#축 2번 레이블 미세조정
ax2.set_ylabel("High-Low Difference")
ax2.tick_params(axis='y')

#범례 두 개
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

#그래프 제목
plt.title("ABNB Stock Analysis: Adjusted Close and High-Low Difference")
plt.show()

## 2. 컷의 미세조정(seaborn)

> 하나 이상의 컷을 만들면 이제부터 소제목과 대제목을 붙일 필요가 생겨난다. 다시 `ds_salaries.csv` 자료를 불러와 여러 컷으로 구성된 페이지를 만들어보자.

In [ ]:
df = pd.read_csv('ds_salaries.csv')

# 1x2 컷이 담긴 페이지
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

fig.suptitle('Main title comes here', fontsize=20)             #대제목

#1번 컷
sns.boxplot(data=df,
            x='company_size', y='salary_in_usd',
            ax=ax[0],
            order=['S','M','L'])
ax[0].set_title('company_size box plot', fontsize=16)          #소제목

#2번 컷
sns.histplot(data=df,
             x='salary_in_usd',
             ax=ax[1])
ax[1].set_title('salary histogram', fontsize=16)               #소제목

> 뭔가 <b>짜임새(layout)</b>가 어색하다. 무엇보다 제목이 너무 가깝다. 이럴 때는 `plt.tight_layout()`을 추가하자.

In [ ]:
# 1x2 컷이 담긴 페이지
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

fig.suptitle('Main title comes here', fontsize=20)             #대제목

#1번 컷
sns.boxplot(data=df,
            x='company_size',
            y='salary_in_usd',
            ax=ax[0],
            order=['S','M','L'])
ax[0].set_title('company_size box plot', fontsize=16)          #소제목

#2번 컷
sns.histplot(data=df,
             x='salary_in_usd',
             ax=ax[1])
ax[1].set_title('salary histogram', fontsize=16)               #소제목

#마법처럼 뭔가 나아진다!
plt.tight_layout()

> 여러 컷을 넣었을 때도 레이아웃이 어색할 수 있다.

In [ ]:
fig, ax = plt.subplots(2, 2)
for idx in range(2):
    for jdx in range(2):
        ax[idx][jdx].set_xlabel('x Label')
        ax[idx][jdx].set_ylabel('y Label')
        ax[idx][jdx].set_title('Title' + str(idx) + str(jdx))

> 원한다면 `plt.tight_layout()`에서도 구체적인 옵션을 줄 수 있다(주지 않을 수도 있다).

In [ ]:
fig, ax = plt.subplots(2, 2)
for idx in range(2):
    for jdx in range(2):
        ax[idx][jdx].set_xlabel('x Label')
        ax[idx][jdx].set_ylabel('y Label')
        ax[idx][jdx].set_title('Title' + str(idx) + str(jdx))

#알맞게 조정해보자
plt.tight_layout(w_pad = 1,       #width
                 h_pad = 4)       #height

> **연습문제 2-1**. `salary`와 `salary_in_usd` 두 변수의 히스토그램을 하나의 그래프 안에 나란히 시각화하시오.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('ds_salaries.csv')
df['log_salary'] = np.log(df['salary'])

# 1x2 컷이 담긴 페이지
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

#salary
sns.histplot(data=df,
            x='log_salary',     #x='salary' 라고 할 수도 있다
            ax=ax[0])

#salary_in_usd
sns.histplot(data=df,
            x='salary_in_usd',
            ax=ax[1])

plt.tight_layout()

## 3. 색채 미세조정(seaborn)

> 특히 산점도의 경우에는 색채를 아름답게 설정해볼 수 있다.

In [ ]:
df = sns.load_dataset('tips')

fig, ax = plt.subplots()
sns.scatterplot(data=df,
                x='total_bill',
                y='tip',
                ax=ax,
                hue='size')

> seaborn에서 `color_palette`을 활용해보자. 첫번째 방식은 사전에 설정된 <b>컬러맵(color map)</b>을 적절히 골라 사용하는 것이다.
>
> ---
> 힌트: 공식 웹사이트를 살펴보자: https://seaborn.pydata.org/generated/seaborn.color_palette.html. 좌측 윈도에서 다른 함수(e.g., `sns.diverging_palette()`) 등도 살펴보자.

In [ ]:
#팔렛1
color = sns.color_palette("light:#006d2c", as_cmap=True)
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='size', palette=color)

#팔렛2
color = sns.color_palette("coolwarm", as_cmap=True)
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='size', palette=color)

#팔렛3
color = sns.diverging_palette(250, 0, as_cmap=True)
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='size', palette=color)

> 아예 특정 팔렛을 지정하거나 각 점에 해당하는 색을 지정할 수도 있다.

In [ ]:
#활용1
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='day', palette='bright')

#활용2
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax,
                hue='day', hue_order=['Thur','Fri','Sat','Sun'],
                palette=['black','cyan','purple','salmon'])

#활용3
prepal1 = {'Thur': 'black',
          'Fri' : 'cyan',
          'Sat' : 'purple',
          'Sun' : 'salmon'}
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='day', palette=prepal1)

#활용4
prepal2 = {'Thur': '#808000',
           'Fri' : '#0000FF',
           'Sat' : '#DDA0DD',
           'Sun' : '#BBF90F'}
fig, ax = plt.subplots()
sns.scatterplot(data=df, x='total_bill', y='tip', ax=ax, hue='day', palette=prepal2)

## 4. 범례의 미세조정(seaborn)

> 이번에 또다른 귀찮은 문제인 <b>범례(legend)</b>의 위치 조정에 관해 살펴보자. 이를 위해 차량별 이산화탄소 배출량 자료를 사용하기로 한다.

In [ ]:
df = pd.read_csv('CO2_Emissions.csv')
df
df.info()

> `Fuel Consumption Comb (L/100 km)`과 `CO2 Emissions(g/km)` 간의 연관성을 시각화한다면 어떻게 하면 좋을까? 단 `Vehicle Class`에 따라 달리 나타내보자.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                hue='Vehicle Class')

> 전혀 알아볼 수가 없다. 그러므로 범례를 우측으로 치우는 편이 좋겠다. 이때는 컷(`ax`)을 수정하게 된다. `bbox_to_anchor` 패러미터에서 수치를 주게 되는데, 좌하단이 (0,0)이고 후상단이 (1,1)이다. 위치를 조정해 가면서 실험해보자.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                hue='Vehicle Class')
ax.legend(bbox_to_anchor=(1.5, 1.5))

> **연습문제 4-1**. 위 그래프를 조금 더 꾸며보자. 이때 `ax.legend()` 안의 패러미터를 변경한다. `loc`과 `ncol`, `fontsize`, `frameon` 등을 추가해보자.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                hue='Vehicle Class',
                alpha=0.6)

#범례를 아래로 옮기고 다중 컬럼으로
ax.legend(bbox_to_anchor=(0.5, -0.15),
          loc='upper center',
          ncol=5,  # 5열로 배치
          fontsize=8,
          frameon=False)

## 5. 문자열 미세조정(seaborn)

> 그림만으로 충분치 않을 때 문자열(text)을 직접 집어넣어 점처럼 시각화할 수도 있다. 그 위치 설정이 제법 골치아픈데 절대위치와 상대위치를 다른 방식으로 지정할 수 있다. 이 수업에서는 일단 절대위치만 다루자. 이때는 실제 $x$축과 $y$축을 보고 지정하면 된다.

In [ ]:
df = pd.read_csv('CO2_Emissions.csv')

fig, ax = plt.subplots()
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                ax=ax,
                hue='Fuel Type')
ax.text(x=10, y=135,         # x축과 y축에서 잘 비교해보자
        s='Ethanol emits less CO2',
        fontdict={'fontsize':10,
                  'weight':'bold'})

> `ax.annotate()`을 사용하면 화살표를 사용할 수 있다. 이때 `xy()`에는 화살표 끝이 가리킬 좌표, `xytext()`는 텍스트가 나올 좌표이다.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                ax=ax,
                hue='Fuel Type')
ax.annotate(text='Ethanol is efficient',
            xy=(26, 400),
            xytext=(20, 200),
            arrowprops={'color':'black',
                        'arrowstyle':'-|>'})   # '-|>', '->', '<-', '|-|', '<->', 'fancy', 'simple'

> **연습문제 5-1**. `Fuel Consumption Comb (L/100 km)`와 `CO2 Emissions(g/km)` 간의 연관성을 시각화하시오. 그 발견의 결과를 텍스트로 표현하시오. 단 아래처럼 `Fuel Type`을 `D`와 `Z`로 제약하고 분석하시오.
>
> ---
> ```python
> df = pd.read_csv('CO2_Emissions.csv')
> df = df.loc[df['Fuel Type'].isin(['D', 'Z'])]
> ```

In [ ]:
df = pd.read_csv('CO2_Emissions.csv')
df = df.loc[df['Fuel Type'].isin(['D', 'Z'])]

fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(data=df,
                x='Fuel Consumption Comb (L/100 km)',
                y='CO2 Emissions(g/km)',
                hue='Fuel Type',
                alpha=0.2)
ax.annotate(text='Diesel emits more CO2 per unit of fuel',
            xy=(9, 250),
            xytext=(10, 150),
            arrowprops={'color':'red',
                        'arrowstyle':'->'})
fig.show()

## 6. 축과 레이블의 미세조정(plotly)

> 지금까지 seaborn을 사용하여 시각화를 연습했다. 다음으로 plotly를 사용하여 똑같은 연습을 해보자.
>
> `ds_salary.csv`를 사용하여 30개 이상의 빈도가 관찰되는 잡 타이틀만 골라내고 시각화해보자. 이때는 겹치지 않도록 자동적으로 회전이 들어간다. 그러나 우리가 직접 인위적으로 조정해보자. `update_xaxes()`를 사용한다.

In [ ]:
import pandas as pd
import plotly.express as px

#연봉 자료
df = pd.read_csv('ds_salaries.csv')
title_over_30 = freq.loc[freq>30]
cond = df['job_title'].isin(title_over_30.index)
df2 = df.loc[cond]

#시각화
fig = px.box(data_frame=df2,
             x='job_title',
             y='salary_in_usd',
             width=1000, height=500)
#fig.update_xaxes(tickfont={'size':15}, tickangle=-90)
fig.show()

> plotly에서 그리드(grid)를 재설정하려면 `update_layout()`를 조절한다. 그림 크기도 좀 조정하자.

In [ ]:
fig = px.box(data_frame=df2,
             x='job_title',
             y='salary_in_usd',
             width=1000, height=500)
fig.update_xaxes(tickfont={'size':15}, tickangle=-90)
#fig.update_layout(yaxis={'showgrid' : False})
fig.show()

> 만일 연봉(`salary`)의 중앙값(median)이 높은 순서대로 `job_title`을 정렬하고, 그 순서대로 시각화한다면 어떻게 할까? plotly에서 그리고 싶은 막대의 순서는 딕트(dict)로 조정한다.

In [ ]:
#딕트로 순서 설정
salary_median = df2.groupby('job_title')['salary'].median()
salary_median

salary_median_sorted = salary_median.sort_values(ascending=False)
salary_median_sorted
salary_median_sorted.index

my_order = {'job_title': salary_median_sorted.index}
my_order

> 이제 `px.box`에서 `category_orders` 패러미터에 그 값을 넣어준다.

In [ ]:
fig = px.box(data_frame=df2,
             y='job_title',
             x='salary_in_usd',
             width=1000, height=500,
             category_orders=my_order)
fig.show()

> 이번엔 plotly에서 시각화 도표를 그려보자. 위와 마찬가지로 plotly에서는 기본적으로 어느 정도 꾸며준다.

In [ ]:
df = pd.read_csv('ABNB_stock.csv')

fig = px.line(data_frame=df,
              x='Date', y='Close',
              width=500, height=400)
fig.show()

> 그러나 plotly에서도 `tickformat` 패러미터를 조정하여 날짜의 포멧을 재설정할 수 있다.

In [ ]:
fig = px.line(data_frame=df,
              x='Date', y='Close',
              width=500, height=400)
#fig.update_xaxes(tickformat='%Y-%m-%d')
#fig.update_xaxes(tickformat='%y-%m-%d')
#fig.update_xaxes(tickformat='%Y')
fig.show()

> 인터렉티블 그래프를 본격적으로 꾸밀 때, plotly는 상당히 많은 기능을 제공한다. 애니메이션을 만들거나, <b>호버(hover)</b>를 꾸밀 수도 있다. 우리가 그 모든 것을 여기서 배우기는 어렵고, 관심있다면 스스로 시간을 투자해보자.

In [ ]:
import plotly.express as px

#dark theme
fig = px.line(data_frame=df,
              x='Date', y='Close',
              width=800, height=500,
              template='plotly_dark')  #또는 'plotly_white', 'seaborn', 'ggplot2'

# 커스텀 색상과 그라데이션
fig.update_traces(line=dict(color='#00D9FF', width=2.5),
                  mode='lines',
                  fill='tozeroy',  # 영역 채우기
                  fillcolor='rgba(0, 217, 255, 0.1)',
                  hovertemplate='<b>날짜</b>: %{x}<br>' + '<b>종가</b>: <i>$%{y}</i><br>'
                  )

fig.show()

> (seaborn에서와 마찬가지로) 물론 plotly를 이용하여 2개의 $y$축 그래프를 그릴 수도 있다.

In [ ]:
#2중축 그래프 작성을 위한 도구
from plotly.subplots import make_subplots

df = pd.read_csv('ABNB_stock.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['High-Low'] = df['High'] - df['Low']

#보조y축 설정
fig = make_subplots(specs=[[{"secondary_y": True}]])

#그림 1
subfig1 = px.line(df, x='Date', y='Close')
subfig1.update_traces(line_color='red')

#그림 2
subfig2 = px.line(df, x='Date', y='Volume')
subfig2.update_traces(line_color='blue')

#두번째 그림의 y축을 y2로 설정
subfig2.update_traces(yaxis='y2')

#그림 두 개 합침!
fig.add_traces(subfig1.data + subfig2.data)

fig.layout.xaxis.title = 'Date'
fig.layout.yaxis.title = 'Close'
fig.layout.yaxis2.title = 'Volume'
fig.layout.yaxis.color = 'red'
fig.layout.yaxis2.color = 'blue'

fig.update_layout(width=1000, height=500)
fig.show()

## 7. 컷의 미세조정(plotly)

> seaborn과는 달리 plotly에서는 한 페이지 안에 여러 컷을 넣는 연습이 조금 어렵다. 그 이유는 (고수준 API인) <b>plotly express (`px`)</b>를 넘어 <b>graph objects (`go`)</b>를 사용해야 하기 때문이다. 이에 관해 좀 더 살펴보자.
>
> 지금까지는 `plotly.express`를 `px`라는 별칭으로 사용해왔다. 그러나 실무에서는 `plotly.graph_objects`를 `go`라는 별칭으로 사용하는 것이 좀 더 일반적인 것 같다. `go`는 `px`에 비해 사용법이 다소 복잡하지만, 훨씬 자유롭게 많은 것들을 할 수 있다. 여러분은 생성형AI가 주로 `go`를 사용하는 것을 깨닫게 될 것이다. 따라서 `go`를 모르면 생성형AI가 내놓는 결과도 제대로 이해하거나 수정할 수 없게 된다.

In [ ]:
#graph object (go)로 작업하기!
import plotly.graph_objects as go

#toy data
xcoord = [i for i in range(0, 10)]
ycoord = [i**2 for i in xcoord]

#빈 도화지와 컷 만들기
fig = make_subplots()

#그림 요소(trace)의 추가. 이때 추가되는 요소는 go인 Scatter
fig.add_trace(
    go.Scatter(x=xcoord,
               y=ycoord,
               mode='lines')     #`lines` = 선 차트, `markers` = 산점도
)

fig.show()

> 실제 데이터로 그림을 그려보자. 거의 비슷하지만 살짝 다르다.

In [ ]:
import plotly.graph_objects as go

#preparing data
df = pd.read_csv('ABNB_stock.csv')
df['Date'] = pd.to_datetime(df['Date'])

fig = make_subplots()

fig.add_trace(
    go.Scatter(x=df['Date'],
               y=df['Close'],
               name='Close',
               mode='lines',
               yaxis='y',
               line={'color':'red'})
)

fig.show()

> `go`를 활용해서 그래프 두 개를 하나의 컷 안에 그려보자.

In [ ]:
import plotly.graph_objects as go

#preparing data
df = pd.read_csv('ABNB_stock.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['High-Low'] = df['High'] - df['Low']

#빈 도화지와 컷 만들기
fig = make_subplots()

#첫번째 그림
fig.add_trace(
    go.Scatter(x=df['Date'],
               y=df['Close'],
               name='Close',
               mode='lines',
               yaxis='y',
               line={'color':'red'})
    )

#두번째 그림
fig.add_trace(
    go.Scatter(x=df['Date'],
               y=df['High-Low'],
               name='High-Low',
               mode='lines+markers',
               yaxis='y2',
               line={'color':'green'})
    )

#레이아웃 업데이트
fig.update_layout(
    xaxis = {'title': "Date"},
    yaxis = {'title': "Close"},
    yaxis2 = {'title': "High-Low",
              'side': "right",
              'anchor': 'x',        #yaxis2를 x축에 연결
              'overlaying': 'y'},   #yaxis2를 y축에 연결
    width=1000, height=500
    )

fig.layout.yaxis.color = 'red'
fig.layout.yaxis2.color = 'blue'

fig.show()

> plotly에서는 어떻게 제목을 설정할까? 이때 호버 효과(hover effect)와 마찬가지로 <i>이탤릭(italic)</i>이나 <b>볼드(bold)</b>와 같이 **마크다운(markdown)** 언어를 사용해 꾸밀 수도 있다. 나름대로 여러분이 제목을 부여해보자.

In [ ]:
df = pd.read_csv('ds_salaries.csv')

#정렬 기준
sort_by = {'company_size': ['S','M','L']}

#box-and-whiskers
fig = px.box(data_frame=df,
             x='company_size', y='salary_in_usd',
             title='<b><i>company_size</i> Box Plot</b>',          #제목
             width=600, height=600,
             category_orders=sort_by)
fig.update_layout({'title_font_size':20})                          #제목 글자 크기
fig.show()

> matplotlib 계열에서 간격 조정은 `update_layout()`을 사용한다. 그러나 plotly에서는 직접 패러미터를 넣어 조정한다.

In [ ]:
df = sns.load_dataset('tips')

fig = px.scatter(data_frame=df,
                 x='total_bill', y='tip',
                 facet_row='sex', facet_col='time',
                 width=600, height=600,
                 facet_row_spacing=0.2,      #이것이 update_layout() 대체
                 facet_col_spacing=0.1)
fig.show()

> **연습문제 7-1**. 원격근무(`remote_ratio`) 비율 별로 연봉(`salary_in_usd`)의 히스토그램을 그린 뒤, 컷의 간격 조정 및 제목 설정을 수행하시오.

In [ ]:
df = pd.read_csv('ds_salaries.csv')
df
df['remote_ratio'].value_counts()

fig = px.histogram(data_frame=df,
                   x='salary_in_usd',
                   title='Salary (in USD) by remote work ratio',   #제목
                   facet_col='remote_ratio',                       #0, 50, 100
                   facet_col_spacing=0.05,
                   width=1200, height=400,
                   category_orders={'remote_ratio':[0, 50, 100]})

fig.show()

## 8. 색채 미세조정(plotly)

> plotly에서도 기본이 되는 몇가지 컬러맵이 있다. 한번 직접 살펴보자.

In [ ]:
fig = px.colors.qualitative.swatches()
fig.show()

fig = px.colors.sequential.swatches_continuous()
fig.show()

fig = px.colors.diverging.swatches_continuous()
fig.show()

fig = px.colors.cyclical.swatches_continuous()
fig.show()

> 직접 구현해보자. 고쳐보면서 마음에 드는 방식으로 꾸며보자.

In [ ]:
df = sns.load_dataset('tips')

fig = px.scatter(data_frame=df, x='total_bill', y='tip', width=500, height=400,
                 color='size',
                 color_continuous_scale='balance')
fig.show()

#discrete sequence color map 1
fig = px.scatter(data_frame=df, x='total_bill', y='tip', width=500, height=400,
                 color='day',
                 color_discrete_sequence=px.colors.qualitative.Light24)
fig.show()

#직접 설정1
prepal3 = {'Thur' : 'black',
           'Fri' : 'cyan',
           'Sat' : 'purple',
           'Sun' : 'salmon'}
fig = px.scatter(data_frame=df, x='total_bill', y='tip', width=500, height=400,
                 color='day',
                 color_discrete_map=prepal3)
fig.show()

#직접 설정2
prepal4 =['rgb(255, 255, 255)',
          'rgb(0, 0, 0)',
          'rgb(128, 128, 128)',
          'rgb(64, 255, 192)']
fig = px.scatter(data_frame=df, x='total_bill', y='tip', width=500, height=400,
                 color='day',
                 color_discrete_sequence=prepal4)
fig.show()

## 9. 범례의 미세조정(plotly)

> 이번엔 plotly에서 범례 위치를 조정해보자. 기본적인 자동 조정이 상당히 뛰어나기 때문에 그다지 고민할 필요가 없다.

In [ ]:
df = pd.read_csv('CO2_Emissions.csv')

fig = px.scatter(data_frame=df,
                 x='Fuel Consumption Comb (L/100 km)',
                 y='CO2 Emissions(g/km)',
                 color='Vehicle Class',
                 width=700, height=500)
#fig.update_layout(legend_x=1.2, legend_y=1)
fig.show()

> **연습문제 9-1**. 시내 주행 연비(`Fuel Consumption City (L/100 km)`와 고속도로 주행 연비(`Fuel Consumption Hwy (L/100 km)`) 간 연관성을 시각화하시오. 이때 연료 유형(`Fuel Type`) 별로 다른 색깔을 부여하시오. 범례를 적절한 위치에 배치하시오.

In [ ]:
fig = px.scatter(data_frame=df,
                 x='Fuel Consumption City (L/100 km)',
                 y='Fuel Consumption Hwy (L/100 km)',
                 color='Fuel Type',
                 opacity=0.2,                   #alpha
                 width=700, height=500)
fig.update_layout(legend_x=0.1, legend_y=1.1,
                  legend_orientation="h")       #horizontal
fig.show()

## 10. 문자열 미세조정(plotly)

> plotly에서는 그냥 `add_annotation()` 하나로 모든 것을 해결한다.

In [ ]:
df = pd.read_csv('CO2_Emissions.csv')

# Plotly annotation
fig = px.scatter(data_frame=df,
                 x='Fuel Consumption Comb (L/100 km)',
                 y='CO2 Emissions(g/km)',
                 width=500, height=400,
                 color='Fuel Type')
fig.add_annotation(x=20,
                   y=130,
                   text='<b>Ethanol emits less CO2</b>',     #bold
                   showarrow=False)
fig.show()

> **연습문제 10-1**. 시내 주행 연비(`Fuel Consumption City (L/100 km)`와 고속도로 주행 연비(`Fuel Consumption Hwy (L/100 km)`) 간 연관성을 시각화하시오. 가장 우상단 끄트머리에 있는 케이스에 대해 `Extremes?'라는 문자열을 배치하시오.

In [ ]:
fig = px.scatter(data_frame=df,
                 x='Fuel Consumption City (L/100 km)',
                 y='Fuel Consumption Hwy (L/100 km)',
                 opacity=.3, width=500, height=500)
fig.add_annotation(x=30, y=21,
                   text='Extremes?',
                   showarrow=True,
                   arrowhead=3)
fig.show()